In [1]:
import pandas as pd

data_train = pd.read_csv("tap_fun_train.csv")
#data_test = pd.read_csv("tap_fun_test.csv")


In [23]:
data_train['register_time_days'] = data_train.register_time.str[:10]

data_train_df = pd.DataFrame(data_train['register_time_days'].value_counts()).sort_index()

# 画图
from pyecharts import Line

line = Line("玩家数量统计-日",title_top="50%")
line.add("玩家数量", data_train_df.index, data_train_df['register_time_days'], 
         mark_line=["average"], mark_point=["max", "min"])

line.render()


In [4]:
import copy  
data_pay_7 = copy.copy(data_train[data_train['pay_price']>0])
print data_pay_7.shape   # (41439, 111)
print (float(data_pay_7.shape[0])/float(data_train.shape[0]))  # 0.018111395638212645


(41439, 110)
0.0181113956382


In [24]:
# ----------------------------- 统计，改列明（避免冲突），合并，计算比例
data_pay_7_day_df = pd.DataFrame(data_pay_7['register_time_days'].value_counts()).sort_index()
# print(data_pay_7_day_df)
data_pay_7_day_df.rename(columns={'register_time_days':'pay_register_time_days'}, inplace = True)
data_day_count = pd.concat([data_pay_7_day_df, data_train_df], axis=1)
# print(data_day_count)
data_day_count['pay_percent'] = data_day_count['pay_register_time_days']/data_day_count['register_time_days']
# print(data_day_count)

# ----------------------------- 画图
from pyecharts import Overlap

line3 = Line()
line3.add("注册玩家数量", data_day_count.index, data_day_count['register_time_days'], mark_line=["average"], mark_point=["max", "min"])

line4 = Line()
line4.add("7天内付费玩家数量", data_day_count.index, data_day_count['pay_register_time_days'], mark_line=["average"], 
          mark_point=["max", "min"], yaxis_max=3000)

overlap = Overlap()
# 默认不新增 x y 轴，并且 x y 轴的索引都为 0
overlap.add(line3)
# 新增一个 y 轴，此时 y 轴的数量为 2，第二个 y 轴的索引为 1（索引从 0 开始），所以设置 yaxis_index = 1
# 由于使用的是同一个 x 轴，所以 x 轴部分不用做出改变
overlap.add(line4, yaxis_index=1, is_add_yaxis=True)
overlap.render()

In [ ]:
chongqian = data_train[data_train['prediction_pay_price']>0]
fenzhong5 = data_train[data_train['avg_online_minutes']<5]
chongqianyouwande = chongqian.merge(fenzhong5 , on = 'user_id')
print chongqianyouwande